# Create TinyGPT via chatgpt
## Prompt

Give a python code for create a model called TinyGPT based on Transformer algo that gets trained on a meaningful text  "It was a bright cold day in April, and the clocks were striking thirteen." Also, give code to save the model, load the model and generate a text based on 10 character prompt.


In [ ]:
!pip install torch

# Define the model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# Set seed for reproducibility
torch.manual_seed(42)

# Hyperparameters
block_size = 32
embedding_size = 64
n_heads = 4
n_layers = 2
dropout = 0.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Sample training text
text = "It was a bright cold day in April, and the clocks were striking thirteen."

# Character-level tokenizer
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long).to(device)

class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embedding_size, head_size, bias=False)
        self.query = nn.Linear(embedding_size, head_size, bias=False)
        self.value = nn.Linear(embedding_size, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.size()
        k = self.key(x)
        q = self.query(x)
        wei = (q @ k.transpose(-2, -1)) / (C**0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        return wei @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, embedding_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_size, 4 * embedding_size),
            nn.ReLU(),
            nn.Linear(4 * embedding_size, embedding_size),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = MultiHeadAttention(n_heads, embedding_size // n_heads)
        self.ff = FeedForward()
        self.ln1 = nn.LayerNorm(embedding_size)
        self.ln2 = nn.LayerNorm(embedding_size)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class TinyGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_size)
        self.pos_embedding = nn.Embedding(block_size, embedding_size)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(embedding_size)
        self.lm_head = nn.Linear(embedding_size, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx)
        pos_emb = self.pos_embedding(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx


# Training the Model

In [2]:
model = TinyGPT().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

def get_batch():
    start = random.randint(0, len(data) - block_size - 1)
    x = data[start:start+block_size].unsqueeze(0)
    y = data[start+1:start+block_size+1].unsqueeze(0)
    return x, y

# Train for a few steps
for step in range(500):
    xb, yb = get_batch()
    logits = model(xb)
    loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(f"Step {step}: Loss {loss.item():.4f}")

Step 0: Loss 3.2469
Step 100: Loss 1.1488
Step 200: Loss 0.6581
Step 300: Loss 0.2267
Step 400: Loss 0.5352


# Save and Load Model

In [3]:
# Save model
torch.save(model.state_dict(), "tinygpt.pth")

# Load model
model_loaded = TinyGPT().to(device)
model_loaded.load_state_dict(torch.load("tinygpt.pth"))
model_loaded.eval()


TinyGPT(
  (token_embedding): Embedding(23, 64)
  (pos_embedding): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x SelfAttentionHead(
            (key): Linear(in_features=64, out_features=16, bias=False)
            (query): Linear(in_features=64, out_features=16, bias=False)
            (value): Linear(in_features=64, out_features=16, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNor

# Generate Text from Prompt

In [4]:
# Prompt (should be 10 characters)
prompt = "bright col"  # Example 10-character prompt
input_idx = torch.tensor([encode(prompt)], dtype=torch.long).to(device)

# Generate 50 characters
output_idx = model_loaded.generate(input_idx, max_new_tokens=50)[0].tolist()
print("Generated text:\n", decode(output_idx))


Generated text:
 bright cold day in April, and the the clocks Aprenockiks tre
